In [2]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
import numpy as np
import cPickle as pickle
import tensorflow as tf
import gym
from matplotlib import pyplot as plt
# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
# gamma = 1-0.
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = True # resume from previous checkpoint?
# resume = False;
render = False
# render = True
backlen=20;

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  I[I == 144] = 0 # erase background (background type 1)
  I[I == 109] = 0 # erase background (background type 2)
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(np.float).ravel()

def discount_rewards(r):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(xrange(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}
def lookback(lst):
    lst = lst[-backlen:];
#     np.pad(lst,(20-lst.size,), 'constant', constant_values=0);
    if len(lst) != backlen:
        lst = [None]*(backlen-len(lst)) + lst;
    return(lst)

In [ ]:
# model initialization
resume = False;
# resume = True;
render = False;
render = True;
H=5;
gamma = 0.99;
D1=80;D2=80;
# signof = lambda x : x
loglik = lambda prob,lab:tf.log(lab*(lab - prob) + (1 - lab)*(lab + prob));


def time_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    grad = 0;
    for t in reversed(xrange(0, r.size)):
#         grad = grad * gamma + r[t] ;
        if r[t] != 0: 
            running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
            grad = 2 * (r[t] > 0) - 1; 
        running_add = running_add + grad;
        discounted_r[t] = running_add
    return discounted_r

D = 80 * 80 # input dimensionality: 80x80 grid
tf.reset_default_graph()
# if resume:
#     pass

# else:
#     observations = tf.placeholder(tf.float32, [None,D] , name="input_x")
#     W1 = tf.get_variable("W1", shape=[D, H],
#                initializer=tf.contrib.layers.xavier_initializer())
#     b1 = tf.constant(0.1, shape=[1,H]);
#     W2 = tf.get_variable("W2", shape=[H, 1],
#                initializer=tf.contrib.layers.xavier_initializer())
#     b2 = tf.constant(0.1, shape=[1,1]);
xinput = tf.placeholder(tf.float32, name="input_x")
observations = tf.reshape(xinput,[-1,D1,D2,1])
conv1 = tf.layers.conv2d(
  inputs=observations,
  filters=H,
  kernel_size=[5, 5],
  padding="same",
  activation=tf.nn.relu)
dense = tf.layers.dense(inputs=tf.reshape(conv1,[-1, D*H]), units=H, activation=tf.nn.relu)
dropout = dense;
#     logits = tf.layers.dense(inputs=dropout, units=10)
# score = tf.layers.dense(inputs=dropout, units=1,activation = tf.nn.relu)

input_y = tf.reshape( tf.placeholder(tf.float32, name="input_y"), [-1,1]);
score = tf.layers.dense(inputs=tf.concat([dense,input_y],1), units=1,activation = tf.nn.relu)
sign =  tf.layers.dense(inputs=tf.concat([dense,input_y],1), units=1,activation = tf.nn.sigmoid)

# tf.nn.sigmoid(score)

tvars = tf.trainable_variables()
# input_y = tf.placeholder(tf.float32,[None,1], name="input_y")
#     advantages = tf.placeholder(tf.float32,name="reward_signal")
rtime = tf.placeholder(tf.float32,name="reward_signal")

#     loglik = tf.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
# loss = tf.reduce_mean( tf.square(score/(tf.abs(rtime)+1) - 1) - tf.sign(sign - tf.sign(rtime)) ); 
loss = tf.reduce_mean( tf.square(score/(tf.abs(rtime)+1) - 1) - loglik(sign,tf.sign(rtime)/2+0.5) ); 

In [50]:
newGrads = tf.gradients(loss,tvars)

batchGrad = [];
for i,var in enumerate(tvars):
    exec('Grad%d'%i + ' = tf.placeholder(tf.float32)');
    exec('batchGrad += [Grad%d]'%i)


optimiser = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
#     batchGrad = optimiser.compute_gradients(loss,tvars);
batchGrad = newGrads;
updateGrads = optimiser.apply_gradients(zip(batchGrad,tvars))
init = tf.global_variables_initializer();

env = gym.make("Pong-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
ys=[];byss=[];rss=[];tpreds=[];
running_reward = None
reward_sum = 0
episode_number = 0

[2017-06-03 16:47:27,010] Making new env: Pong-v0


In [184]:
# xinput.shape

TensorShape(None)

In [42]:
# all_vars
# tf.get_tensor_by
sess.close()

In [52]:
ModelName = 'signabs_RL_pong_RMSprop'
# ModelName = 'expolike_RL_pong_RMSprop';
ModelFile = 'Models/'+ModelName+'.ckpt';
render = False;
resume = 1;
batch_size=1;
learning_rate = 10e-2

# ModelName='save.ckpt'
# fname = 
# with tf.Session() as sess:
# gdcmd='grad_dict = {'+','.join(['Grad%d: gradBuffer[%d]'%(i,i) for i,k in enumerate(tvars)])+'}';
while True:
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True));
    print(sess.run(tf.constant('init')))
#     gradBuffer = [np.zeros_like(v) for v in sess.run(tvars)]
#     grad_buffer = { k : np.zeros_like(v) for k,v in model.iteritems() } # update buffers that add up gradients over a batch
    if resume: 
#         tf.train.import_meta_graph(ModelFile+'.meta')    
        oSaver = tf.train.Saver()
#         oSaver.restore(sess,ModelFile)
#         oSaver = tf.train.import_meta_graph(ModelFile+'.meta')    
        oSaver.restore(sess,ModelFile)
    else:
        sess.run(init)
    while True:
        if render: env.render()

        # preprocess the observation, set input to network to be difference image
        cur_x = prepro(observation)
        diff_x = cur_x - prev_x if prev_x is not None else np.zeros(D)
        prev_x = cur_x
        x = np.reshape(diff_x,[1,D1,D2,1]);
#         aprob = sess.run(probability,feed_dict={observations: x})
        aprob = 0.5;
        action = 2 if np.random.uniform() < aprob else 3 # roll the dice!
        y = 1. if action == 2 else 0. # a "fake label"
        tpred = sess.run(score,feed_dict={observations: x, input_y:np.array([[y]])})    

        # record various intermediates (needed later for backprop)
        xs.append(x) # observation
        ys.append(y)
        tpreds.append(tpred);

        # step the environment and get new measurements
        observation, reward, done, info = env.step(action)
        reward_sum += reward
        drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

        if done: # an episode finished
            episode_number += 1

            # stack together all inputs, hidden states, action gradients, and rewards for this episode
            epx = np.vstack(xs)
            epy = np.vstack(ys);
            epr = np.vstack(drs)
            eptpred=np.vstack(tpreds);
            xs,hs,dlogps,drs,ys,tpreds = [],[],[],[],[],[] # reset array memory
            
#             # compute the discounted reward backwards through time
#             discounted_epr = discount_rewards(epr)
#             # standardize the rewards to be unit normal (helps control the gradient estimator variance)
#             discounted_epr -= np.mean(discounted_epr)
#             discounted_epr /= np.std(discounted_epr)
#             discounted_epr = discount_rewards(epr);
            time_epr=time_rewards(epr);
#            curr_loss = sess.run(loss,feed_dict={xinput: epx, input_y: epy, rtime: time_epr});
            lst =  sess.run([loss,updateGrads],feed_dict={xinput: epx, input_y: epy, rtime: time_epr});
            curr_loss = lst[0];
#             curr_loss = sess.run(loss,feed_dict={xinput: epx, input_y: epy, rtime: time_epr});
            
#             tGrad = sess.run([loss,newGrads,updateGrads],feed_dict={xinput: epx, input_y: epy, rtime: time_epr});
#             for ix,grad in enumerate(tGrad):
#                 gradBuffer[ix] += grad
            
            # perform rmsprop parameter update every batch_size episodes
            if episode_number % batch_size == 0:
                pass
#                 grad_dict={varname:}
#                 exec(gdcmd);
#                 sess.run(updateGrads,feed_dict={Grad0: gradBuffer[0],W2Grad:gradBuffer[1]})
#                 sess.run(updateGrads)
#                 sess.run(updateGrads,feed_dict={newGrads:tGrad})
#                 sess.run(updateGrads,feed_dict={xinput: epx, input_y: epy, rtime: time_epr});
             
                
#                 gradBuffer = [np.zeros_like(v) for v in sess.run(tvars)]
    
            # boring book-keeping
            running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
            print 'resetting env. episode %d reward total was %f. loss_func: %f' % (episode_number, reward_sum, curr_loss)
            if episode_number % 10  == 9: 
                oSaver = tf.train.Saver()
                oSess = sess
                oSaver.save(oSess, ModelFile) 
#                 pickle.dump(tf, open('save.p', 'wb'))
            reward_sum = 0
            observation = env.reset() # reset env
            prev_x = None

        if reward != 0: # Pong has either +1 or -1 reward exactly when game ends. 
            bys=lookback(ys);
            byss.append(bys);
            rss.append(reward);
            if len(byss)-1 == 100:
                    byss.pop(0);
                    rss.pop(0);

            pass;
    #     print ('ep %d: game finished, reward: %f' % (episode_number, reward)) + ('' if reward == -1 else ' !!!!!!!!')

init
INFO:tensorflow:Restoring parameters from Models/signabs_RL_pong_RMSprop.ckpt


[2017-06-03 17:11:16,229] Restoring parameters from Models/signabs_RL_pong_RMSprop.ckpt


resetting env. episode 133 reward total was -20.000000. loss_func: 1.201498
resetting env. episode 134 reward total was -21.000000. loss_func: 1.088402
resetting env. episode 135 reward total was -20.000000. loss_func: 1.284551
resetting env. episode 136 reward total was -21.000000. loss_func: 1.099302
resetting env. episode 137 reward total was -21.000000. loss_func: 1.089321
resetting env. episode 138 reward total was -21.000000. loss_func: 1.072787
resetting env. episode 139 reward total was -21.000000. loss_func: 1.060967
resetting env. episode 140 reward total was -20.000000. loss_func: 1.180430
resetting env. episode 141 reward total was -21.000000. loss_func: 1.062440
resetting env. episode 142 reward total was -21.000000. loss_func: 1.057021
resetting env. episode 143 reward total was -21.000000. loss_func: 1.048258
resetting env. episode 144 reward total was -21.000000. loss_func: 1.067431
resetting env. episode 145 reward total was -20.000000. loss_func: 1.159281
resetting en

KeyboardInterrupt: 